'''
THIS FILE IS FOR TRAINING THE TREE PHENOLOGY MODEL
CODE WRITTEN FOR CUBESAT MANTIS PROGRAM (DSS)
AUTHOR: CHAZ DAVIES
''' 


In [3]:

import tensorflow as tf
import numpy as np
import matplotlib as img 
import rasterio
import os
import image_process as ip
from rasterio.plot import show
import json
import keras


ModuleNotFoundError: No module named 'image_process'

In [ ]:

path = 'C:\\Users\\chazd\\OneDrive\\Desktop\\git\\DSS\\DSS-MANTIS-OBC-AI\\Images\\algal-bloom'

In [ ]:
## set constants
# might change during training process
image_height = 160
image_width = 175
bands = 4


In [ ]:
# Make array to store label names

labels = {
        "discard" : 0, # image not relevant
        "HAB-present" : 1, # harmful algal bloom detected in region
        "no-HAB-present" : 2, # no harmful algal bloom detected in region
    }


labels

{'discard': 0, 'HAB-present': 1, 'no-HAB-present': 2}

In [2]:
# copy the path
tiff_files = ip.loadDataSet(path)
file_path = ["" for x in range(len(tiff_files))]

# concactinate the folder path to the file name
for i in range(len(tiff_files)):
    file_path[i] = os.path.join(path, tiff_files[i])


NameError: name 'ip' is not defined

In [6]:
#extract dates
dates = ["" for x in range(len(file_path))]
for i, file in enumerate(file_path):
    dates[i] = file[len(file)-15:len(file)-5]



In [7]:
raster_imgs = np.empty(len(file_path)) 

#open each file
img = rasterio.open(file_path[1])

    


c:\Users\chazd\AppData\Local\Programs\Python\Python39\lib\site-packages\rasterio\__init__.py:331: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [8]:
# Open images and move them to a numpy array
batch = len(file_path)
# set dimension of 3d-array to image height, width and number of bands
# ---- maybe add a dimension for labels

unlabeled_images = np.zeros((batch, image_height, image_width, bands))

for i in range(batch):
    #open raster image
    img = rasterio.open(file_path[i])

    for j in range(bands):
        unlabeled_images[i,:,:,j] = img.read(j+1)



In [9]:
# Load JSON file with image file names and their corresponding labels
with open('C:\\Users\\chazd\\OneDrive\\Desktop\\git\\DSS\\DSS-MANTIS-OBC-AI\\Images\\trees_feb2020-Dec2021_10m_labels.json') as json_file:
    data = json.load(json_file)

# Extract filenames and annotations
file_names = [item["filename"] for item in data]
annotations_from_json = [item["annotations"] for item in data]

# Map annotations from JSON to numerical labels using the label dictionary
numerical_labels = np.array([labels[annotation] for annotation in annotations_from_json])


## Preprocessing

Before prepocessing the images, a quick review of the dataset

In [10]:
print("Dimensions: ", unlabeled_images.shape)
print("Batch Size: ", len(unlabeled_images))

Dimensions:  (66, 160, 175, 4)
Batch Size:  66


In [11]:
# normalize all values
import cv2
import numpy as np

def normalize_4channel_image(image):
    # Split the image into Red, Green, Blue, and NIR channels
    r, g, b, nir = cv2.split(image)

    # Normalize each channel
    r_normalized = (r - np.min(r)) / (np.max(r) - np.min(r))
    g_normalized = (g - np.min(g)) / (np.max(g) - np.min(g))
    b_normalized = (b - np.min(b)) / (np.max(b) - np.min(b))
    nir_normalized = (nir - np.min(nir)) / (np.max(nir) - np.min(nir))

    # Recombine the channels
    normalized_image = cv2.merge([r_normalized, g_normalized, b_normalized, nir_normalized])

    return normalized_image



In [12]:
normalized= unlabeled_images

for i in range(len(unlabeled_images)):
    
    normalized[i] = normalize_4channel_image(unlabeled_images[i])
    


In [13]:
#split into test and train
split = np.random.randint(0,len(normalized), size = int(len(normalized)*0.9))

train_images = normalized[split]
train_labels = numerical_labels[split]
test_images = normalized[-split]
test_labels = numerical_labels[-split]


## Building Model

In [14]:
from tensorflow.keras import datasets, layers, models

In [21]:
model = models.Sequential()

model.add(layers.Conv2D(16,(3,3),1,activation= 'relu', input_shape=(160,175,4)))
model.add(layers.MaxPooling2D())

model.add(layers.Conv2D(32,(3,3),1, activation= 'relu'))
model.add(layers.MaxPooling2D())

model.add(layers.Conv2D(16,(3,3),1, activation= 'relu'))
model.add(layers.MaxPooling2D())

# ry softmax

model.add(layers.Flatten())
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(5, activation='softmax')) 

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 158, 173, 16)      592       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 79, 86, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 77, 84, 32)        4640      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 38, 42, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 36, 40, 16)        4624      
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 18, 20, 16)       

In [22]:
#compile
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])



## Train Model

In [23]:
history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

Epoch 1/10


c:\Users\chazd\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


2/2 [==============================] - 5s 1s/step - loss: 1.5231 - accuracy: 0.2034 - val_loss: 1.4345 - val_accuracy: 0.1525
Epoch 2/10
2/2 [==============================] - 1s 310ms/step - loss: 1.3100 - accuracy: 0.5424 - val_loss: 1.2552 - val_accuracy: 0.6102
Epoch 3/10
2/2 [==============================] - 0s 271ms/step - loss: 1.2550 - accuracy: 0.5424 - val_loss: 1.1626 - val_accuracy: 0.6102
Epoch 4/10
2/2 [==============================] - 0s 275ms/step - loss: 1.2007 - accuracy: 0.5424 - val_loss: 1.1490 - val_accuracy: 0.6102
Epoch 5/10
2/2 [==============================] - 0s 292ms/step - loss: 1.1278 - accuracy: 0.5424 - val_loss: 1.0990 - val_accuracy: 0.6102
Epoch 6/10
2/2 [==============================] - 0s 271ms/step - loss: 1.0620 - accuracy: 0.5424 - val_loss: 1.0531 - val_accuracy: 0.6102
Epoch 7/10
2/2 [==============================] - 0s 274ms/step - loss: 1.0067 - accuracy: 0.5424 - val_loss: 0.9739 - val_accuracy: 0.6102
Epoch 8/10
2/2 [==================

## Model Evaluation

In [24]:
import matplotlib.pyplot as plt

# plt.plot(history.history['accuracy'], label='accuracy')
# plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.ylim([0.5, 1])
# plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)


2/2 - 0s - loss: 0.8584 - accuracy: 0.6780 - 113ms/epoch - 57ms/step


In [25]:
print("Accuracy = ", test_acc*100, "%")

Accuracy =  67.79661178588867 %
